In [3]:
import geomesa_pyspark

from pyspark.sql import Row
conf = geomesa_pyspark.configure().setAppName('Demo1')
 
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext

# geomesa_pyspark.init_sql(spark) in later version 2.4 DOCS)
spark._jvm.org.apache.spark.sql.SQLTypes.init(spark._jwrapped)

In [7]:
filename = "file:///home/jovyan/DEMO/gdf_polygon_1200.csv"

df = spark.read \
    .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(filename)

In [8]:
df.first()

Row(_c0=0, TARGET_FID=58152, suburb_loc='Karori', Combustibl=1, SHAPE_Leng=91.54329378070867, SHAPE_Area=268.44508528391873, AU2013Num=575000, IgnProb_bl=0.000506607709467, RandProb=0.887780267427702, geometry='POLYGON ((1745723.379300117 5428052.979199886, 1745713.707950115 5428050.243400097, 1745712.449649811 5428054.691549778, 1745704.59800005 5428052.470449924, 1745705.789549828 5428048.258200169, 1745698.611899853 5428046.227799892, 1745696 5428055.461050034, 1745701.088399887 5428056.90045023, 1745699.486949921 5428062.561749935, 1745708.888700008 5428065.221300125, 1745710.544050217 5428059.369400024, 1745720.754600048 5428062.257750034, 1745723.379300117 5428052.979199886))', area=268.44508528391873, centroid='POINT (1745708.762695227 5428056.083294064)', X=1745708.7626952266, Y=5428056.083294064, d_short=0.0, d_long='inf')

In [61]:
alteredDF = df \
    .withColumn("polygon", st_polygonFromText('geometry'))
#                 \
#     .withColumn("centroid", geomesa_pyspark.st_makePoint("X", "Y"))



AttributeError: RasterFrames have not been enabled for the active session. Call 'SparkSession.withRasterFrames()'.

In [53]:
#from https://github.com/locationtech/rasterframes/blob/43bd3b37f0b2931470b2d5b5757474d3c885e659/pyrasterframes/src/main/python/pyrasterframes/rasterfunctions.py#L718-L1062


# from __future__ import absolute_import
# from pyspark.sql.column import Column, _to_java_column
# from pyspark.sql.functions import lit
# # from .rf_context import RFContext
# # from .rf_types import CellType, Extent, CRS

# def _apply_column_function(name, *args):
# #     jfcn = RFContext.active().lookup(name)
#     jcols = [_to_java_column(arg) for arg in args]
#     return Column(*jcols)

# def st_polygonFromText(*args):
#     """"""
#     return _apply_column_function('st_polygonFromText', *args)


In [140]:
from pyrasterframes.utils import create_rf_spark_session
from pyrasterframes.rasterfunctions import *
# spark = create_rf_spark_session()
# st_polygonFromText
import pyspark.sql.functions as fn

In [142]:
import geomesa_pyspark as gpk

In [148]:
help(gpk.spark.GeoMesaSpark.apply)

Help on function apply in module geomesa_pyspark.spark:

apply(self, params)



In [80]:
class MyContext(object):
    """
    Entrypoint to RasterFrames services
    """
    def __init__(self, spark_session):
        self._spark_session = spark_session
        self._gateway = spark_session.sparkContext._gateway
        self._jvm = self._gateway.jvm
        #jsess = self._spark_session._jsparkSession
        #self._jrfctx = self._jvm.org.locationtech.rasterframes.py.PyRFContext(jsess)
        self.context = self._jvm.org.locationtech.geomesa.spark.jts.DataFrameFunctions
        
    def lookup(self, function_name: str):
        return getattr(self._jrfctx, function_name)        


In [81]:
ctx = MyContext(spark).context

In [127]:
cl = spark.sparkContext._gateway.jvm.org.locationtech.geomesa.spark.jts.udf.SpatialRelationFunctions

In [136]:
member = spark.sparkContext._gateway.jvm.org.locationtech.geomesa.spark.jts.udf.GeometricConstructorFunctions.ST_MakeBBOX

In [139]:
member(1.0,1.0,2.0,3.1)

Py4JError: An error occurred while calling z:org.locationtech.geomesa.spark.jts.udf.GeometricConstructorFunctions.ST_MakeBBOX. Trace:
py4j.Py4JException: Method ST_MakeBBOX([class java.lang.Double, class java.lang.Double, class java.lang.Double, class java.lang.Double]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)

